In [1]:
import os
# import model as modellib
# import unet_new as modellib2
# import vgg16unet
import argparse
import numpy as np
import tensorflow as tf
import pandas as pd
import os
import h5py
from data_defores_generator import data_gen, create_list_id,data_gen_not_batch
from datetime import datetime
from tensorflow.keras.callbacks import (ModelCheckpoint, TensorBoard, CSVLogger, History, EarlyStopping, LambdaCallback,ReduceLROnPlateau)

from tensorflow.keras import layers, backend, Model, utils
from matplotlib import pyplot as plt
import rasterio
import cv2
from tensorflow.compat.v1.keras.backend import set_session
# import wanrings
# warnings.filterwarnings("ignore")
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
set_session(tf.compat.v1.Session(config=config))
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

2022-11-18 10:47:25.158359: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-18 10:47:25.172248: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-18 10:47:25.231969: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-18 10:47:25.232111: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [2]:
FMT_VALMODEL_PATH ="{}_val_weights.h5"
FMT_VALMODEL_LAST_PATH = "{}_val_weights_last.h5"
FMT_VALMODEL_HIST = "{}_val_hist.csv"
PREFIX = "checkpoints"
INPUT_CHANNEL =  300
INPUT_SIZE = 512
early_stop = 50
num_class = 1
weight_path = r""
log_dir = r"/home/geoai/eodata/cloud_detect_tensorflow/cloud_only_v2/logs"
num_epoch = 200
batch_size= 16
scale_neg = 0.1

In [3]:
from model import build_model
unet = build_model((512,512,4), 42) #36??
# unet = modellib2.build_model((None,None,3), 32)

# lowest size (None, 64, 64, 512)

2022-11-18 10:47:27.255977: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-18 10:47:27.256331: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-18 10:47:27.256579: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-18 10:47:27.257080: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-18 10:47:27.257331: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

In [4]:
weight_path = r"/home/geoai/eodata/cloud_detect_tensorflow/cloud_only_v2/cp_new.h5"
if weight_path != "":
    unet.load_weights(weight_path)

In [5]:
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

In [6]:
unet.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 4  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 42  1554        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 512, 512, 42  168        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [7]:
checkpoint_dir = os.path.join(log_dir, 'weights')
os.makedirs(checkpoint_dir, exist_ok=True)
model_checkpoint = ModelCheckpoint(
    os.path.join(checkpoint_dir, FMT_VALMODEL_PATH.format(PREFIX + "_{epoch:04d}")),
    verbose=1,
    save_best_only=False,
    save_weights_only=False)

In [8]:
model_earlystop = EarlyStopping(
    monitor='val_loss',   #val_jaccard_coef_int
    patience=early_stop,
    verbose=0,
    mode='auto'
)

In [9]:
model_history = History()

model_board = TensorBoard(
    log_dir=os.path.join(log_dir, 'logs'),
    histogram_freq=0,
    write_graph=True,
    embeddings_freq=0)

In [10]:
lr_reducer = ReduceLROnPlateau(
factor=np.sqrt(0.1),
verbose=1,
patience=30,
min_lr=0.5e-7)

In [11]:
fig_dir = os.path.join(log_dir, 'figs')
os.makedirs(fig_dir, exist_ok=True)

def visualize_weight(epoch, model, image_path, mask_path, pos_val):
    INPUT_SIZE = 512
    color = (0,0,0,0)
    list_id = pos_val
    indx= np.random.choice(len(list_id),4)
    x=[]
    y=[]
    for idx in indx: 
        im_name = list_id[idx]
        image_fn = os.path.join(image_path, im_name)
        mask_fn = os.path.join(mask_path, im_name)
        with rasterio.open(image_fn, 'r') as f1:
            width,height = f1.width,f1.height
            new_image_width = new_image_height = max(width,height)
            # if width < 512 and height < 512:
            values = f1.read().transpose(1,2,0)
            x_center = (new_image_width - width) // 2
            y_center = (new_image_height - height) // 2
            result = np.full((new_image_height,new_image_width, 4), color, dtype=np.float16)
            result[y_center:y_center+height, x_center:x_center+width] = values ## REMOVE indices if input 3 channels
            
            if result.dtype == np.float16: pass
            else: result = cv2.resize(result,(INPUT_SIZE, INPUT_SIZE), interpolation = cv2.INTER_CUBIC)
            image = result
            
            with rasterio.open(mask_fn, 'r') as f:
                mask = f.read().astype(np.uint8)
                mask = np.array((mask[0]==2).astype(np.uint8)) #255
                result_mask = np.full((new_image_height,new_image_width), 0, dtype=np.uint8)
                result_mask[y_center:y_center+height, x_center:x_center+width] = mask
                result_mask = cv2.resize(result_mask,(INPUT_SIZE, INPUT_SIZE), interpolation = cv2.INTER_CUBIC).astype(np.uint8)
#                 mask = np.expand_dims(result_mask,axis = -1)
                mask = result_mask
            x.append(image)
            y.append(mask)
    y_predict = model.predict(np.array(x)/np.finfo(np.float16).max)
    ressult_predict = []
    for i in range(len(x)):        
        predict_mask = y_predict[i].reshape((INPUT_SIZE,INPUT_SIZE))
        predict_mask = (predict_mask>0.5).astype(np.uint8)
        ressult_predict.append(predict_mask)
    fig = plt.figure(figsize=(7, 7))
    rows = 4
    columns = 3
    count = 0
    # if np.random.randint(0, 1000) < 100:
    for i in range(len(x)):
        count = count+1
        image = x[i]
        fig.add_subplot(4, 3, count)
        im_converted = np.array((image/np.finfo(np.float16).max)*255, dtype=np.uint8)
        plt.imshow(im_converted) ##
#         plt.axis('off')
        plt.title("image_{}".format(str(i)))
        mask = y[i]
        count = count+1
        fig.add_subplot(4, 3, count)
        plt.imshow(np.array(mask,dtype = float), cmap='gray')
#         plt.axis('off')
        plt.title("mask_{}".format(str(i)))
        predict = ressult_predict[i]
        count = count+1
        fig.add_subplot(4, 3, count)
        plt.imshow(np.array(predict,dtype = float), cmap='gray')
#         plt.axis('off')
        plt.title("result{}".format(str(i)))
    plt.savefig(os.path.join(fig_dir,'line plot{}_epoch{}.jpg'.format(PREFIX,epoch+1)), dpi=200)
    plt.close(fig)


In [12]:
cwd = r"/home/geoai/eodata/cloud_detect_tensorflow/cloud_only_v2"
image_path = os.path.join(cwd,r"data/train/image")
mask_path = os.path.join(cwd,r"data/train/label")

val_image_path = os.path.join(cwd,r"data/val/image")
val_mask_path = os.path.join(cwd,r"data/val/label")

In [13]:
image_list = create_list_id(image_path)

image_list_val = create_list_id(val_image_path)

np.random.shuffle(image_list)
count = len(image_list)    
# cut_idx = int(round(count*split_ratio))    
# train_list = image_list[0:cut_idx]
# val_list = [id_image for id_image in image_list if id_image not in train_list]
train_list = image_list
val_list = image_list_val

visualize_epoch = LambdaCallback(on_epoch_end=lambda epoch, logs: visualize_weight(epoch, unet, image_path, mask_path, train_list))

In [14]:
num_chanel = INPUT_CHANNEL
batch_size = 2
num_epoch = 600
unet.fit_generator(
    generator= data_gen_not_batch(image_path, mask_path, train_list, augment = True, batch_size = batch_size),
    validation_data = data_gen_not_batch(val_image_path, val_mask_path, val_list, augment = False, batch_size = batch_size),
    initial_epoch = 300,
    validation_steps = 43,#30,#161
    steps_per_epoch = 260,#135, #len(image_list)//batch_size, ##2
    epochs=num_epoch,
    verbose = 1,
    callbacks=[model_checkpoint, model_history, model_board, lr_reducer, visualize_epoch],#model_earlystop
    max_queue_size=20
)

/tmp/ipykernel_5229/3984764091.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  unet.fit_generator(
/home/skymap/anaconda3/envs/tf2.7/lib/python3.8/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 301/600


2022-11-18 10:47:34.880438: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8500
2022-11-18 10:47:36.105951: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-11-18 10:47:36.107132: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.6
2022-11-18 10:47:36.107144: W tensorflow/stream_executor/gpu/asm_compiler.cc:233] Used ptxas at ptxas
2022-11-18 10:47:36.107189: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be 

260/260 [==============================] - ETA: 0s - loss: 0.0699 - accuracy: 0.9770 - f1_m: 0.9494 - precision_m: 0.9771 - recall_m: 0.9257 - dice_coef: 0.9599
Epoch 00301: saving model to /home/geoai/eodata/cloud_detect_tensorflow/cloud_only_v2/logs/weights/checkpoints_0301_val_weights.h5
260/260 [==============================] - 140s 499ms/step - loss: 0.0699 - accuracy: 0.9770 - f1_m: 0.9494 - precision_m: 0.9771 - recall_m: 0.9257 - dice_coef: 0.9599 - val_loss: 0.0464 - val_accuracy: 0.9839 - val_f1_m: 0.9549 - val_precision_m: 0.9831 - val_recall_m: 0.9336 - val_dice_coef: 0.9633 - lr: 1.0000e-05
Epoch 302/600
260/260 [==============================] - ETA: 0s - loss: 0.0632 - accuracy: 0.9785 - f1_m: 0.9510 - precision_m: 0.9755 - recall_m: 0.9292 - dice_coef: 0.9616
Epoch 00302: saving model to /home/geoai/eodata/cloud_detect_tensorflow/cloud_only_v2/logs/weights/checkpoints_0302_val_weights.h5
260/260 [==============================] - 126s 485ms/step - loss: 0.0632 - accura

In [15]:
unet.save(os.path.join(log_dir, FMT_VALMODEL_LAST_PATH.format(PREFIX)))

: 